<h2>Análise das classificações</h2>

<i>Após a aquisição das classificações feitas pelos usuários, este código possui o intuito de analisá-las estatisticamente.</i>

In [ ]:
import BaseDados
import collections
import numpy as np
import matplotlib.pyplot as plt
import itertools
import jsonify
from sklearn.metrics import f1_score,confusion_matrix
from matplotlib.colors import ListedColormap

#https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
def plot_confusion_matrix(cm, classes, title='Matriz Confusão', cmap=None):
    cm = cm.astype('float') / cm.sum(axis=0)[np.newaxis, :]
        
    N = 256
    vals = np.ones((N, 4))
    newcmp = ListedColormap(vals)
    
    plt.imshow(cm, interpolation='nearest', cmap=newcmp)
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else '.0f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="black", weight='bold' if i==j else "normal")

    plt.ylabel('Ground-Truth')
    plt.xlabel('Predicted')
    plt.tight_layout()

<i>O código abaixo é utilizado para analisar o score de cada usuário quando o mais votado for considerado como ground-truth, excluindo as classificações com empate</i>

In [ ]:
#if c.Identificador not in [1541786078,1165676880,1331336016]
cls = [c for c in BaseDados.GetLetraClassificacoes() if c.Identificador not in [1] ];
#Cada Identificador
#Cada Musica Classificada por ele
#Pega todas as avaliações da música(valencia / assunto) tirando a dele e pega qual teve a maior votacao
#Se nao teve maior votada alertar que nao foi possivel definir
f1s_val = []; # F1 Score por usuário (Valência)
f1s_ass = []; # F1 Score por usuário (Assunto)
ids = []; #Identificadores dos usuários
qtds = []; #Quantidade classificada por usuário
cfns_val = []; #Matriz de confusão por usuário (Valência)
cfns_ass = []; #Matriz de confusão por usuário (Assunto)
indes_val = []; #Letras que tiveram empate na classificação (Valência)
indes_ass = []; #Letras que tiveram empate na classificação (Assunto)
for id in set([l.Identificador for l in cls]):
    ids.append(id);
    vals_mc = []; #Opções mais votadas (valência) para cada letra classificada pelo usuário
    vals_id = []; #Opção do usuário (valência)
    asss_mc = []; #Opções mais votadas (Assunto) para cada letra classificada pelo usuário
    asss_id = [];#Opção do usuário (Assunto)
    qtds.append(len([l.Cod_Letra for l in cls if l.Identificador == id]));
    for codL in set([l.Cod_Letra for l in cls if l.Identificador == id]):
        avals_val = [l.Valencia for l in cls if l.Cod_Letra == codL];
        avals_ass = [l.Assunto for l in cls if l.Cod_Letra == codL];
        mcs_val = [[c[0],c[1]] for c in collections.Counter(avals_val).most_common()];
        mcs_ass = [[c[0],c[1]] for c in collections.Counter(avals_ass).most_common()];
        if (len(mcs_val)>1 and mcs_val[0][1] == mcs_val[1][1]):
            indes_val.append(codL);
        if (len(mcs_ass)>1 and mcs_ass[0][1] == mcs_ass[1][1]):
            indes_ass.append(codL);
        if not (len(mcs_val)>1 and mcs_val[0][1] == mcs_val[1][1]) :
            vals_mc.append(mcs_val[0][0]);
            vals_id.append([l.Valencia for l in cls if l.Identificador == id and l.Cod_Letra == codL][0]);
        if  not(len(mcs_ass)>1 and mcs_ass[0][1] == mcs_ass[1][1]) :
            asss_mc.append(mcs_ass[0][0]);
            asss_id.append([l.Assunto for l in cls if l.Identificador == id and l.Cod_Letra == codL][0]);
    print(id,",",vals_mc,",",vals_id,",",len([l.Cod_Letra for l in cls if l.Identificador == id]));
        
    if (len(vals_mc) > 0): 
        f1s_val.append(f1_score(vals_mc, vals_id, average='macro'));
        cfn_vals = confusion_matrix(vals_mc, vals_id);
        if cfn_vals.size==4:
            cfn_vals = np.resize(cfn_vals,(3,3));
        cfns_val.append(cfn_vals);
        
    if (len(asss_mc) > 0): 
        f1s_ass.append(f1_score(asss_mc, asss_id, average='macro'));
        cfn_asss = confusion_matrix(asss_mc, asss_id);
        if cfn_asss.size==4:
            cfn_asss = np.resize(cfn_asss,(3,3));
        cfns_ass.append(cfn_asss);
        
print(set(indes_val))
print(set(indes_ass))

<i>O código abaixo apresenta os resultados (F1 de cada usuário) em valência, assunto e quantas letras o mesmo classificou</i>

In [ ]:
for i in range(0,len(f1s_val)):
    print(ids[i],"\t",f1s_val[i],"\t",f1s_ass[i],"\t",qtds[i])
    
plt.plot(range(0,len(qtds)),qtds)

<i>Media de F1 (Assunto)</i>

In [ ]:
print(np.array(f1s_ass).mean())
print(np.array(f1s_ass).std())

<i>Media de F1 (Valência)</i>

In [ ]:
print(np.array(f1s_val).mean())
print(np.array(f1s_val).std())

<i></i>

In [ ]:
sum = 0
for i in range(0,len(qtds)):
    sum += qtds[i]
    
print(len(qtds))

<i>Matriz de confusão(Valência)</i>

In [ ]:
plt.figure(figsize=(3,3))
cfnt = [[0,0,0],[0,0,0],[0,0,0]]
for c in cfns_val:
    cfnt+= c;
plot_confusion_matrix(cfnt,['Pos.','Neu.','Neg.'],1,'');

<i>Matriz de confusão(Assunto)</i>

In [ ]:
plt.figure(figsize=(3,3))
cfnt = [[0,0,0],[0,0,0],[0,0,0]]
for c in cfns_ass:
    cfnt+= c;
plot_confusion_matrix(cfnt,['Relat.','Reflec.','Others'],1,'')

<i>Conversão para JSON</i>

In [ ]:
base = BaseDados.SQLiteConn();
res = base.execute('''select l.Artista,l.Nome,REPLACE(l.Letra,'"','') Letra,group_concat(REPLACE(REPLACE(REPLACE(lc.Assunto,'3','Others'),'2','Emotions/Reflections'),'1','Relationship')) Assunto,group_concat(REPLACE(REPLACE(REPLACE(lc.Assunto,'3','Negative'),'2','Neutral'),'1','Positive')) Valencia  from Letras l ,Letra_Classificacoes lc
where l.Cod_Letra = lc.Cod_Letra
group by l.Cod_Letra''').fetchall();
json.dumps([dict(row) for row in res]);